In [21]:
# импортирую библиотеки
import pandas as pd
import os
import gdown
from datetime import datetime, date
from sklearn import preprocessing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
uploaded = files.upload()   
df = pd.read_csv(list(uploaded.keys())[0])   # в df лежит основной датафрейм, где отображены основные данные участников

Saving ╨г╤З╨░╤Б╤В╨╜╨╕╨║╨╕ anonimized.csv to ╨г╤З╨░╤Б╤В╨╜╨╕╨║╨╕ anonimized (42).csv


Загружаем папку с данными по соревнованиям, на выходе получим два списка, которые будут содержать информацию за индивидуальные и групповые этопы.

In [23]:
file_list = os.listdir()
all_data = []
for file_name in file_list:
    if file_name.endswith('.csv'):
        df1 = pd.read_csv(file_name)
        all_data.append(df1)
indiv_data = []    # в этой переменной лежат все датафреймы, в которых отображены результаты за ИНДИВИДУАЛЬНЫЕ этапы
group_data = []    # в этой переменной лежат все датафреймы, в которых отображены результаты за ГРУППОВЫЕ этапы
for i in all_data:
  if ('ФИО' in i.columns) and ('Баллы, %' in i.columns):
    indiv_data.append(i)
  elif ('Баллы, ед.' in i.columns):
    group_data.append(i)

Обработаем данные за индивидуальные этапы и на выходе получим новую колонку в исходном датафрейме, которая показывает баллы за индивидуальный зачет.

In [24]:
i_d = []
for i in indiv_data:
  new_df = i.iloc[:, [1,4]]
  i_d.append(new_df)
i_d.pop(9)
for i in i_d:
  i.iloc[:, 1] = i.iloc[:, 1].str.replace(',', '.')
for i in i_d:
  merged_table = pd.merge(df, i, on='ФИО', how = 'left')
  df = merged_table
df.iloc[:, 15:37] = df.iloc[:, 15:37].fillna(0.0)
df.iloc[:, 15:37] = df.iloc[:, 15:37].astype(float)
df['individual_result'] = df.iloc[:, 15:37].sum(axis=1)
df = df.drop(df.columns[15:37], axis=1)

<ipython-input-24-fc85329e2003>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i.iloc[:, 1] = i.iloc[:, 1].str.replace(',', '.')
<ipython-input-24-fc85329e2003>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i.iloc[:, 1] = i.iloc[:, 1].str.replace(',', '.')
<ipython-input-24-fc85329e2003>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

Добавим новую метрику 'Перспективность', которая исчисляется как отношение индивидуально набранных баллов и стажа работы.

In [25]:
today = pd.to_datetime(date.today())
df['Дата'] = pd.to_datetime(df['Начало трудового стажа'], format='%Y/%m/%d', errors='coerce')
df['Стаж'] = (today - (df['Дата']))
df['Стаж'] = df['Стаж'].astype(str)
df['Стаж'] = df['Стаж'].fillna(0)
df['Стаж'] = df['Стаж'].str.extract('(\d+)')
df['Стаж'] = df['Стаж'].astype(float) / 30
df['Перспективность'] = df['individual_result'] / (df['Стаж']) 
df.drop(['Стаж', 'Дата'], axis = 1, inplace = True)

Нормируем данные в столбце перспективность от 0 до 1, а также создадим отдельный столбец, в котором пометим перспективных представителей единицей.

In [26]:
df['Перспективность'] = (df['Перспективность'] - df['Перспективность'].min()) / (df['Перспективность'].max() - df['Перспективность'].min())
df = df.rename(columns={'Перспективность': 'Скор/стаж'})
df['Перспективность'] = df['Скор/стаж'].apply(lambda x: 1 if x > 0.25 else 0)   

Создадим в исходной таблице столбец 'Возраст'.

In [27]:
today = pd.to_datetime(date.today())
df['Дата_р'] = pd.to_datetime(df['Дата рождения'], format='%Y/%m/%d', errors='coerce')
df['Возраст'] = (today - (df['Дата_р']))
df['Возраст'] = df['Возраст'].astype(str)
df['Возраст'] = df['Возраст'].fillna(0)
df['Возраст'] = df['Возраст'].str.extract('(\d+)')
df['Возраст'] = round(df['Возраст'].astype(float) / 365, 0)
df.drop(['Дата_р'], axis = 1, inplace = True)

Добавим в таблицу с информацией по групповым этапам баллы соответствующих групп.

In [28]:
g_d = []
for i in group_data:
  new_df = i.iloc[:, [1,2,3,5]]
  g_d.append(new_df)
for i in g_d:
  i['ФИО'] = i['ФИО участников'].str.split(';')
data = g_d[0]
for i in range(1, len(g_d)):
  data = pd.concat([data, g_d[i]])
data.iloc[:, [3,5]] = data.iloc[:, [3,5]].fillna(0.0)
data.iloc[:, 3] = data.iloc[:, 3].str.rstrip('%')
data.iloc[:, 5] = data.iloc[:, 5].str.rstrip('%')
data.iloc[:, 3] = data.iloc[:, 3].str.replace(',','.')
data.iloc[:, 5] = data.iloc[:, 5].str.replace(',','.')
data.iloc[:, [3,5]] = data.iloc[:, [3,5]].astype(float)
data.iloc[:, [3,5]] = data.iloc[:, [3,5]].fillna(0.0)
data['group_result'] =  data['Баллы, ед.'] + data['Баллы, %']
data.drop(['ФИО участников', 'Баллы, %', 'Баллы, ед.'], axis = 1, inplace = True)

<ipython-input-28-f7c2178f7726>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['ФИО'] = i['ФИО участников'].str.split(';')
<ipython-input-28-f7c2178f7726>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['ФИО'] = i['ФИО участников'].str.split(';')
<ipython-input-28-f7c2178f7726>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Создадим в исходной таблице 2 дополнительные метрики участников: 'Количество средних задач', 'Количество сложных задач'. Они являются абсолютной величиной количества решенных задач участниками и исчисляются относительно успешности решений конкурентов.

In [29]:
best = []
ok = []
for w in indiv_data:
  top = []
  norm = []
  sr = []
  for i in w.columns:
    if w[i].nunique() == 1:
        sr.append(i)
  w = w.drop(sr, axis = 1)
  w = w.dropna(axis = 1)
  cv = w.std() / w.mean() * 100
  cv_sorted = cv.sort_values(ascending = False)
  rank_top = list(cv_sorted[0:3].keys())
  for j in rank_top:
    top.append(list(w[w[j] == w[j].max()]['ФИО'].values))
  rank_norm = list(cv_sorted[3:7].keys())
  for k in rank_norm:
    norm.append(list(w[w[k] == w[k].max()]['ФИО'].values))
  for u in top:
    for y in u:
      best.append(y)
  for u1 in norm:
    for y1 in u1:
      ok.append(y1)
id_counts = pd.Series(best).value_counts()
df_best = pd.DataFrame({'id': id_counts.index, 'Количество': id_counts.values})
id_counts_1 = pd.Series(ok).value_counts()
df_ok = pd.DataFrame({'id': id_counts_1.index, 'Количество': id_counts_1.values})
df_best = df_best.drop(index=0)
df_ok = df_ok.drop(index=0)
df_best = df_best.rename(columns={'id': 'ФИО'})
df_ok = df_ok.rename(columns={'id': 'ФИО'})
df = pd.merge(df, df_best, on='ФИО', how = 'left')
df = pd.merge(df, df_ok, on='ФИО', how = 'left')
df = df.rename(columns={'Количество_x': 'Количество сложных задач', 'Количество_y': 'Количество средних задач'})

<ipython-input-29-20eb78a1845c>:12: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cv = w.std() / w.mean() * 100
<ipython-input-29-20eb78a1845c>:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cv = w.std() / w.mean() * 100
<ipython-input-29-20eb78a1845c>:12: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  

По аналогии с индивидуальным зачетом, создадим столбцы 'Количество средних задач', 'Количество сложных задач' для групповых этапов.

In [30]:
best_1 = []
ok_1 = []
for w in group_data:
  top_1 = []
  norm_1 = []
  sr_1 = []
  for i in w.columns:
    if w[i].nunique() == 1:
        sr_1.append(i)
  w = w.drop(sr_1, axis = 1)
  w = w.dropna(axis = 1)
  cv = w.std() / w.mean() * 100
  cv_sorted = cv.sort_values(ascending = False)
  rank_top_1 = list(cv_sorted[0:3].keys())
  for j in rank_top_1:
    top_1.append(list(w[w[j] == w[j].max()]['Название команды'].values))
  rank_norm_1 = list(cv_sorted[3:7].keys())
  for k in rank_norm_1:
    norm_1.append(list(w[w[k] == w[k].max()]['Название команды'].values))
  for u in top_1:
    for y in u:
      best_1.append(y)
  for u1 in norm_1:
    for y1 in u1:
      ok_1.append(y1)
id_counts = pd.Series(best_1).value_counts()
df_best = pd.DataFrame({'Название команды': id_counts.index, 'Количество': id_counts.values})
id_counts_1 = pd.Series(ok_1).value_counts()
df_ok = pd.DataFrame({'Название команды': id_counts_1.index, 'Количество': id_counts_1.values})
df_best = df_best.drop(index=0)
df_ok = df_ok.drop(index=0)
data = pd.merge(data, df_best, on='Название команды', how = 'left')
data = pd.merge(data, df_ok, on='Название команды', how = 'left')
data = data.rename(columns={'Количество_x': 'Количество сложных задач', 'Количество_y': 'Количество средних задач'})

<ipython-input-30-93f7687a0f14>:12: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cv = w.std() / w.mean() * 100
<ipython-input-30-93f7687a0f14>:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cv = w.std() / w.mean() * 100
<ipython-input-30-93f7687a0f14>:12: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  

In [34]:
df

,ФИО,Пол,Роль в мероприятии,Список компетенций,Должность,Категория,Начало трудового стажа,Место работы,Профессия,Дата рождения,...,Образование,Место образования,Год оканчания,Специальность,individual_result,Скор/стаж,Перспективность,Возраст,Количество сложных задач,Количество средних задач
0,ФИО_0,0,Участник мероприятия,Цифровое ПСР-Предприятие (Lean Smart Plant);,Инженер по подготовке производства,Инженер,1993-06-01,АО Чепецкий механический завод,Промышленная автоматика,1975-04-14,...,NaN,NaN,NaN,NaN,0.00,0.000000,0,48.0,NaN,NaN
1,ФИО_1,1,Участник мероприятия,Управление качеством;,Главный специалист,NaN,2000-08-01,Акционерное общество Научно-исследовательский...,NaN,1977-07-19,...,NaN,NaN,NaN,NaN,0.00,0.000000,0,46.0,NaN,NaN
2,ФИО_2,0,Участник мероприятия,Сварочные технологии;,Электрогазосварщик,NaN,NaN,"""Филиал Акционерного общества """"Русатом Инфрас...",NaN,NaN,...,NaN,NaN,NaN,NaN,13.78,NaN,0,NaN,NaN,NaN
3,ФИО_3,0,Участник мероприятия,Инженер-конструктор;,инженер-конструктор,NaN,июл.05,"""Акционерное Общество """"Машиностроительный зав...",NaN,23.06.1986,...,NaN,NaN,NaN,NaN,38.48,NaN,0,NaN,2.0,NaN
4,ФИО_4,0,Участник мероприятия,Геодезия;,Мастер-лаборант,NaN,NaN,ОБПОУ Курский монтажный техникум,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,ФИО_708,0,Участник мероприятия,Бетонные строительные работы;,Бетонщик,NaN,NaN,ООО Трест Рос Спец Энерго монтаж,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00,NaN,0,NaN,NaN,NaN
709,ФИО_709,1,Участник мероприятия,Управление качеством;,Инженер по качеству,NaN,09.2013,"""ФГУП """"Приборостроительный завод""""""",NaN,22.06.1990,...,NaN,NaN,NaN,NaN,0.00,NaN,0,NaN,NaN,NaN
710,ФИО_710,1,Участник мероприятия,Аналитический контроль;,Лаборант химического анализа,NaN,2016-10-10,Центральная научно-исследовательская лаборатория,NaN,1993-02-01,...,Высшее,ИрГУПС,2016,Эксплуатация железных дорог,50.09,0.139654,0,30.0,NaN,1.0
711,ФИО_711,0,Участник мероприятия,Токарные работы на станках с ЧПУ;,токарь,NaN,NaN,"""АО """"Опытное Конструкторское Бюро Машинострое...",NaN,NaN,...,NaN,NaN,NaN,NaN,56.56,NaN,0,NaN,NaN,NaN


In [35]:
data

,Название команды,Компетенция,ФИО,group_result,Количество сложных задач,Количество средних задач
0,АЭХК,Вывод из эксплуатации объектов использования а...,"[ФИО_218, ФИО_489, ФИО_490, ФИО_602]",68.2,2.0,NaN
1,ГХК,Вывод из эксплуатации объектов использования а...,"[ФИО_293, ФИО_458, ФИО_460, ФИО_611]",65.57,2.0,NaN
2,ИАТЭ МИФИ,Вывод из эксплуатации объектов использования а...,"[ФИО_99, ФИО_112, ФИО_467, ФИО_548]",40.31,1.0,NaN
3,Маяк-1,Вывод из эксплуатации объектов использования а...,"[ФИО_368, ФИО_433, ФИО_462, ФИО_683]",82.07,2.0,NaN
4,Маяк-2,Вывод из эксплуатации объектов использования а...,"[ФИО_135, ФИО_189, ФИО_349, ФИО_542]",55.99,2.0,NaN
...,...,...,...,...,...,...
143,МАЯК_1,Инженерное проектирование,"[ФИО_24, ФИО_56, ФИО_130, ФИО_700]",5.0,2.0,NaN
144,АЭП_1_ЛюBIMки,Инженерное проектирование,"[ФИО_85, ФИО_296, ФИО_446, ФИО_486]",54.385,NaN,NaN
145,АЭП_2_Енотики_и_точка,Инженерное проектирование,"[ФИО_28, ФИО_331, ФИО_400, ФИО_589]",54.71,NaN,NaN
146,МИФИ,Инженерное проектирование,"[ФИО_227, ФИО_235, ФИО_334, ФИО_597]",5.0,NaN,NaN
